**DATASET DOWNLOAD**

In [ ]:
import zipfile
import os

# Define path to the zip file
zip_path = "/content/archive.zip"

# Define path to extract to
extract_path = "/content/"

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed.")


Extraction completed.


**IMPORTS**

In [ ]:
# Core Python & Utilities
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm

# TensorFlow & Keras
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import set_global_policy

# Metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Mixed Precision for A100
set_global_policy("mixed_float16")

# Model save path
model_path = "cricket_model.keras"


**DATA AUGMENTATION  &  PREPROCESSING**

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import preprocess_input

# Dataset configuration
data_dir = "Cricket Legends/Cricket Legends"
batch_size = 256
img_size = (150, 150)

# Load training and validation datasets
train_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_dataset.class_names
print(class_names)

# ✅ Save number of classes before .map()/.prefetch() changes structure
num_classes = len(train_dataset.class_names)


val_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

# Preprocess images using MobileNet's expected input format
def preprocess(image, label):
    return preprocess_input(image), label

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)

# Optimize pipeline performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)


Found 6948 files belonging to 30 classes.
Using 5559 files for training.
['adam_gilchrist', 'alastair_cook', 'allan_donald', 'brian_lara', 'chris_gayle', 'curtly_ambrose', 'dale_steyn', 'don_bradman', 'garfield_sobers', 'glenn_mcgrath', 'ian_botham', 'imran_khan', 'james_anderson', 'jaques_kallis', 'kapil_dev', 'kumar_sangakkara', 'mahela_jayawardene', 'ms_dhoni', 'muttiah_muralitharan', 'rahul_dravid', 'richard_hadlee', 'ricky_ponting', 'sachin_tendulkar', 'shane_warne', 'shoaib_akhtar', 'steve_waugh', 'sunil_gavaskar', 'virat_kohli', 'viv_richards', 'wasim_akram']
Found 6948 files belonging to 30 classes.
Using 1389 files for validation.


**LOADING SAVED MODEL IF PRESENT**

In [ ]:
if os.path.exists(model_path):
    print("✅ Found saved model. Loading it and continuing training...")

    # Load saved model
    model = tf.keras.models.load_model(model_path)


else:
  print("❌ No saved model found. Skipping training. You need to define and train a new model.")

✅ Found saved model. Loading it and continuing training...


**READY TO TRAIN USING A PRE-TRAINED MODEL**

**TRANSFER LEARNING DEFINATION: Transfer learning is when you take a model that was trained on a large dataset (like ImageNet)and reuse its learned knowledge for a different but related task — usually with less data and time.**

**TRANSFER LEARNING IMPLEMENTATION**


**1. Load Mobilenet Without its Top Layer**


**2. Freeze the convolutional base so its weights arent updated during training**


**3. Add a custom Classification Layer**


**4. Compile and Train**

In [ ]:
# Loading MobileNet Model Without Top Layer


base_model = MobileNet(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)

<ipython-input-4-466915571>:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


**FREEZE THE CONVULUTIONAL BASE LAYER**

In [ ]:
base_model.trainable = False

**ADDING CUSTOM CLASSIFICATION LAYER**

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),

    layers.Dense(512),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),

    layers.Dense(256),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.4),

    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),

    layers.Dense(num_classes, activation='softmax')
])



**COMPILE**

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

**IF SAVED MODEL PRESENT START FROM HERE**

**TRAINING**

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 399ms/step - accuracy: 0.9792 - loss: 0.0715 - val_accuracy: 0.7279 - val_loss: 1.8590
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9788 - loss: 0.0766 - val_accuracy: 0.7293 - val_loss: 1.8906
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9718 - loss: 0.0832 - val_accuracy: 0.7243 - val_loss: 1.9281
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9756 - loss: 0.0790 - val_accuracy: 0.7178 - val_loss: 1.9429
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9750 - loss: 0.0774 - val_accuracy: 0.7250 - val_loss: 1.9074
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9774 - loss: 0.0682 - val_accuracy: 0.7257 - val_loss: 1.8846
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9749 - loss: 0.0787 - val_accuracy: 0.7207 - val_loss: 1.9113
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9783 - loss: 0.0720 - val_accuracy: 

**SAVE MODEL AFTER TRAINING**

In [ ]:
model.save(model_path)

**EVALUATION METRICS**

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Get true labels and predictions
y_true = []
y_pred = []

for images, labels in val_dataset:
    preds = model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Precision
precision = precision_score(y_true, y_pred, average='weighted')
print("Precision:", precision)

# Recall
recall = recall_score(y_true, y_pred, average='weighted')
print("Recall:", recall)

# F1 Score
f1 = f1_score(y_true, y_pred, average='weighted')
print("F1 Score:", f1)


Accuracy: 0.7228221742260619
Precision: 0.731880610871191
Recall: 0.7228221742260619
F1 Score: 0.7211219510601874
